In [8]:
!pip3 install pandas fuzzywuzzy
!pip3 install matplotlib
!pip3 install pandas


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.8 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 19.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.7 MB 14.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 21.4 MB/s eta 0:00:01
     |████████████████████████████████| 249 kB 25.2 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 10.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m 

In [9]:
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime
import matplotlib.pyplot as plt
from dateutil import parser
import json

Matplotlib is building the font cache; this may take a moment.


In [10]:
file = './IB_Goldens.csv'
golden = pd.read_csv(file).fillna("NA")
original_comparison = pd.read_csv('./241105_merged_results (11).csv').fillna("NA").replace('', 'NA').replace('None', 'NA')
validations = pd.read_csv("./241105_merged_checkpoint_result (8).csv")

columns_to_check = ['broker_name', 'broker_email', 'broker_telephone',
       'broker_postcode', 'broker_agency_name', 'customer_name',
       'customer_postcode', 'business_description', 'trade_description',
       'renewal_date', 'target_premium', 'response_by', 'current_insurer',
       'product', 'existing_broker', 'case_rec_date']

In [11]:
golden = golden[golden['case'] != 'NA']

In [12]:
golden.tail()

,case,broker_agency_name,broker_email,broker_name,broker_postcode,broker_telephone,business_description,case_rec_date,current_insurer,customer_name,...,target_premium,trade_description,el_cover_required,el_geog_limit,el_indemnity_limit,el_indemnity_limit_clause,el_ern,el_wageroll_json,el_wageroll_table_sv_json,el_wageroll_nontable_sv_json
261,201792,Marsh Ltd,Yvonne.Ehinmoti@marsh.com,Yvonne Ehinmoti,RG6 1PT,+44 (0)7392 122111,"Property Owners and Operators or Restaurant, C...",24/05/2023,NA,"Auriens Ltd, Auriens Operations Ltd, Auriens M...",...,NA,Entertainment & Leisure,Yes,"Great Britain, Northern Ireland, The Isle of M...",10000000.0,"Data Protection Act Compensation Awards, Defen...",NA,{},{},{}
262,201793,Towergate Insurance Brokers,matt.capon@towergate.co.uk,Matt Capon,NN4 7XD,+44 (0)1536 486736,Online Furniture Store,NA,NA,NCF Furnishings Limited,...,48000.0,Retail - Online,Yes,NA,10000000.0,NA,NA,{},{},{}
263,201794,Jaggi & Co. Ltd,ian@jaggi.co.uk,Ian Henehan,TW7 4DA,+44 (0)2085 603100,Indoor virtual driving range Potential to be u...,15/06/2023,NA,MyClubHaus Ltd,...,5555.0,Entertainment & Leisure,Yes,NA,10000000.0,NA,NA,{},{},{}
264,201795,REICH Group of Companies,isaac.lowe@reichinsurance.co.uk,Isaac Lowe,M3 5EQ,+44 (0)1618 305461,"Manufacture, distribution and installation of ...",26/06/2023,NA,Signscape and Signconex Limited,...,NA,Manufacturing - not elsewhere classified,Yes,Worldwide,10000000.0,PS100 third party property damage excess incre...,NA,{},{},{}
265,201796,Basil Fry & Company,raven@basilfry.co.uk,Raven Bruce,KT22 8AH,+44 (0)1372 385985,Removal & Storage Contractors and Self-Storage...,16/05/2023,RSA,R&D Schofield Limited,...,16115.49,Storage,Yes,NA,10000000.0,NA,NA,{},{},{}


In [116]:
validations.columns

Index(['broker_name', 'broker_email', 'broker_telephone', 'broker_type',
       'broker_address', 'broker_postcode', 'broker_agency_name',
       'customer_name', 'customer_address', 'customer_postcode',
       'business_description', 'trade_description', 'renewal_date',
       'target_premium', 'response_by', 'current_insurer', 'product',
       'existing_broker', 'case_rec_date', 'el_wageroll_table', 'case', 'date',
       'email_subject'],
      dtype='object')

In [117]:
validations = validations[columns_to_check + ['case'] ]
validations


,broker_name,broker_email,broker_telephone,broker_postcode,broker_agency_name,customer_name,customer_postcode,business_description,trade_description,renewal_date,target_premium,response_by,current_insurer,product,existing_broker,case_rec_date,case
0,True,True,True,False,True,True,False,True,True,False,True,False,True,True,True,False,201512
1,True,True,True,False,False,True,True,True,False,True,False,False,False,True,False,False,201513
2,True,True,True,False,False,False,True,True,False,True,False,False,True,True,False,True,201514
3,True,True,True,False,False,True,True,True,False,True,True,False,False,True,False,False,201515
4,False,False,False,True,True,True,True,True,False,False,False,False,True,True,True,True,201516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,True,True,True,False,False,True,True,True,False,True,False,True,True,True,True,True,201792
262,True,True,True,False,False,True,False,True,False,True,True,True,False,True,True,False,201793
263,True,True,True,False,True,True,False,True,False,False,True,False,True,False,True,True,201794
264,True,True,True,False,False,True,True,True,False,False,False,False,True,True,True,False,201795


In [118]:
import ast

def normalize_columns(value):
    k =  ast.literal_eval(value)
    # print(value, type(value), k)
    return k[0]

comparison = original_comparison[columns_to_check + ['case'] ].copy()

for column in columns_to_check:
    comparison[column] = original_comparison[column].apply(normalize_columns).copy()

# df_copy['column_name'] = df_copy['column_name'].apply(lambda x: ast.literal_eval(x)[0] if isinstance(x, str) else x)
comparison = comparison.fillna("NA").replace('', 'NA').replace('None', 'NA')

In [119]:
pd.set_option('display.max_columns', None)

comparison.head()

,broker_name,broker_email,broker_telephone,broker_postcode,broker_agency_name,customer_name,customer_postcode,business_description,trade_description,renewal_date,target_premium,response_by,current_insurer,product,existing_broker,case_rec_date,case
0,Dan Davis,dan.davis@momentumsolutions.co.uk,+44 (0)1162 388880,LE19 4XT,Momentum Broker Solutions Limited,UE Group Ltd,NA,NA,NA,NA,33000.0,21/06/2023,Axa,NA,yes,16/06/2023,201512
1,Emma Parker,emma_parker@ajg.com,7849574075,SO16 9JX,Gallagher,Beau Monde Estate Ltd,PO32 6BD,Hoteliers & Property Owners,Hoteliers & Property Owners,28/06/2023,NA,NA,NA,Combined,NA,NA,201513
2,Andrea Keenan,andrea@spenceinsurance.co.uk,+44 (0)1506 539230,EH48 2TQ,Spence Insurance,Foodbank West Lothian,EH48 2EP,Foodbank,Charity,31/05/2023,NA,NA,Unknown,Commercial Combined,NA,17/05/2023,201514
3,Lisa Forshaw,LForshaw@guardianib.co.uk,+44 (0)1942 244215,WN3 6XE,GRP Retail Limited,Galloways Bakers Limited,WN3 4ET,Bakery and Retail of Bakery Products from Own ...,Retail,23/06/2023,37000.0,15/06/2023,NA,Commercial Combined,NA,12/06/2023,201515
4,Abigail Hale,abigail.hale@partnersand.com,+44 (0)7917 768426,HP13 6NU,Partners& Limited,Nedging Hall Estate Limited,CO10 2YA,Brewery & Public House with accomadation,Retail - including repair,01/06/2023,NA,23/05/2023,National Farmers Union Mutual and Avon Insuran...,Commercial Combined,National Farmers Union Mutual and Avon Insuran...,23/05/2023,201516


In [120]:
comparison['broker_telephone'] = comparison['broker_telephone'].astype(str)
golden['broker_telephone'] = golden['broker_telephone'].astype(str)

comparison['case'] = comparison['case'].astype(float)
golden['case'] = golden['case'].astype(float)
validations['case'] = validations['case'].astype(float)

In [121]:
# date format: dd/mm/yyyy
comparison.tail()

,broker_name,broker_email,broker_telephone,broker_postcode,broker_agency_name,customer_name,customer_postcode,business_description,trade_description,renewal_date,target_premium,response_by,current_insurer,product,existing_broker,case_rec_date,case
261,Yvonne Ehinmoti,Yvonne.Ehinmoti@marsh.com,+44 (0)7392 122111,RG6 1PT,Marsh McLennan,UK Senior Livings Holdings Limited,SW3 2ST,"Restaurant, Cinema and Gym Facilities",Property Owner,04/06/2023,NA,23/05/2023,Allianz,Commercial Combined,Allianz,24/05/2023,201792.0
262,Matt Capon,matt.capon@towergate.co.uk,+44 (0)1536 486736,NN4 7XD,Towergate Insurance Brokers,NCF Furnishings Limited,B46 1AQ,Furniture Store,Retail,05/06/2023,48000.0,29/05/2023,NA,Commercial Combined,yes,NA,201793.0
263,Ian Henehan,ian@jaggi.co.uk,+44 (0)2085 603100,TW7 4DA,Jaggi & Co. Ltd,MyClubHaus Ltd,RH2 7DH,Indoor virtual driving range. Potential to be ...,Retail,NA,5555.0,NA,TBC,NA,Yes,15/06/2023,201794.0
264,Isaac Lowe,isaac.lowe@reichinsurance.co.uk,+44 (0)1618 305461,M3 5EQ,REICH Group of Companies,Signscape And Signconex Ltd,BS40 5DJ,"Manufacture, distribution and installation of ...",Manufacturing - basic metals and fabricated me...,30/06/2024,NA,NA,Reich Insurance,Combined Package,Reich Insurance,26/06/2023,201795.0
265,Basil Fry & Company,basil.fry@removersbroker.com,NA,CO11 2LR,BASIL FRY & COMPANY,R&D Schofield Limited,CO11 2LR,Contractors and Operators,Storage,31/05/2023,NA,05/05/2023,NA,Commercial Combined,NA,NA,201796.0


In [122]:
validations.head()

,broker_name,broker_email,broker_telephone,broker_postcode,broker_agency_name,customer_name,customer_postcode,business_description,trade_description,renewal_date,target_premium,response_by,current_insurer,product,existing_broker,case_rec_date,case
0,True,True,True,False,True,True,False,True,True,False,True,False,True,True,True,False,201512.0
1,True,True,True,False,False,True,True,True,False,True,False,False,False,True,False,False,201513.0
2,True,True,True,False,False,False,True,True,False,True,False,False,True,True,False,True,201514.0
3,True,True,True,False,False,True,True,True,False,True,True,False,False,True,False,False,201515.0
4,False,False,False,True,True,True,True,True,False,False,False,False,True,True,True,True,201516.0


In [123]:
len(set(comparison['case']) - set(golden['case']))

0

In [124]:
len(set(golden['case']) - set(comparison['case']))

0

In [125]:
len(set(validations['case']) - set(comparison['case']))

0

In [126]:
set(validations['case']) - set(comparison['case'])

set()

In [127]:
set(comparison['case']) - set(validations['case'])

set()

In [128]:
if set(validations['case']) - set(comparison['case']) or set(comparison['case']) - set(validations['case']):
    raise Exception("Validations and Comparisons datasets must not contain mismatching cases!")

In [129]:
set(golden['case']) - set(comparison['case'])

set()

In [130]:


'2023-05-12 00:00:00'
golden_date_format = "%Y-%m-%d %H:%M:%S"
comparison_date_format = "%d/%m/%Y"

In [131]:
def update_cells(data, to_update,  update_value=False):
    
    for index, row in to_update.iterrows():
        comparison_row = data[data['case']== row['case']]
        
        # print(comparison_row)
        # print(row)
        
        for column in to_update.columns:
            try:
                comparison_value = comparison_row[column].values[0]
            except:
                print(row)
                print(comparison_row)
                continue

            is_na = str(comparison_value).lower() in ['na', 'n/a', 'nan', 'null', 'none', 'not urgent', 'contact the broker for renewal date']
            
            if is_na or pd.isna(comparison_value) :
                # print("updated at validations at ", index, column, comparison_row['case'].values[0])
                to_update.at[index, column] = update_value

            if (is_na or pd.isna(comparison_value) ) and column=='product' :
                print("updated at validations at ", index, column, comparison_row['case'].values[0])
                # to_update.at[index, column] = update_value

    return to_update


# validations = update_cells(comparison, validations, update_value=False)

In [132]:
def fuzzy_match(value1, value2):
    return fuzz.ratio(value1, value2)


def clean_str(val):
    symbols = ['£', ",", "'", '"', 'PS']
    val_copy = val
    
    for symbol in symbols:
        val_copy = val_copy.replace(symbol, "")
    
    return val_copy.strip()


def parse_kilo_to_val(value):
    
    return float( value.split('k')[0]) * 1000


def match_target_premium(value1, value2):
    if 'liaise' in value1.lower():
        if 'NA' in value2:
            return True
        return False

    value1 = clean_str(str(value1)).lower()
    if '-' in value1:
        value1 = value1.split('-')[1]
    if '/' in value1:
        value1 = value1.split('/')[1]

    try:
        if 'k' in value1:
            value1 = parse_kilo_to_val(value1)
    
        value2 = clean_str(str(value2)).lower()
    
        if 'k' in value2:
            value2 = parse_kilo_to_val(value2)

        value1 = float(value1)
        value2 = float(value2)
    except: 
        return False
    
    match = value1 == value2
    # if not match:
    #     print(value1, value2, False, type(value1), type(value2))
    return value1==value2

In [133]:
# possible_strings_in_dates = ["na", "not urgent", 'contact the broker for renewal date', "contact the broker for renewal date\'"]

def parse_and_format_golden_date(value):

    if isinstance(value, datetime):
        return value.strftime(comparison_date_format)
    
    parsed = None
    if not '/' in value:
        print('non date value in golden ',value)
        return value    
    try:
        parsed = datetime.strptime(value.strip(), '%d/%m/%Y')
    except Exception as e:
        try:
            parsed = datetime.strptime(value.strip(), '%d/%m/%y')
        except Exception as e2:
            print("g", e, e2)
            return value

    converted = parsed.strftime(comparison_date_format)    
    return str(converted)


# def format_comparison_date(value):
#     if isinstance(value, str):
#         try:    
#             value = datetime.strptime(value, comparison_date_format)
#         except Exception as e:
#             # print("c", e)
#             return value

#     return str(value.strftime(comparison_date_format))


In [134]:
def format_comparison_date(date_str):
    if date_str is None:
        return 'None'
    if not '/' in date_str or not '-' in date_str:
        return date_str
    try:
        datetime_obj = datetime.strptime(date_str.strip(), '%d/%m/%Y')
        return f"{datetime_obj.strftime('%d/%m/%Y')}"
    except ValueError as e:
        print(e)
        return f"{date_str}"

In [135]:
def custom_callback(column_name, value1, value2):
    try:
        fuzzy_colnames = ['broker_name', 'broker_agency_name', 'customer_name', 'trade_description', 'existing_broker', 'business_description']
        date_colnames = ['case_rec_date', 'renewal_date', 'response_by']

        value2 = not value2.empty and value2.values[0] or 'NA'

        if 'na' in (value1, value2) or 'NA' in (value1, value2):
            if value1 == value2: #True Negative
                return True 
            
            return False # False Negative/Positive
        
        if column_name in fuzzy_colnames:
            if value1 is None:
                if value2 is None:
                    return True
                return False
            score = fuzzy_match(value1.lower(), value2.lower())
            return score > 80
            
        if column_name == 'target_premium':
            return match_target_premium(str(value1), str(value2))

        if column_name in date_colnames:
            try:
            #     if not isinstance(value2, datetime): # golden
            #         value2 = datetime.strptime(value2.strip(), '%d/%m/%Y')
            #     if not isinstance(value1, datetime):
            #         value1 = datetime.strptime(value1.strip(), '%d/%m/%Y')
            # except:
            #     print("date err", column, value1, value2)
            #     return value1 == value2
            
                first_match_try = datetime.strptime(value1.strip(), '%d/%m/%Y') == datetime.strptime(value2.strip(), '%d/%m/%Y')
                
                if not first_match_try:
                    return datetime.strptime(value1.strip(), '%m/%d/%Y') == datetime.strptime(value2.strip(), '%d/%m/%Y')
                
                return first_match_try
            except Exception as e:
                print("date err", column, value1, value2, value1 == value2, e)
                return value1 == value2

        if column_name == 'broker_telephone':
            
            if '.' in value1: # value 1 is sometiems parsed as "float" string, remove the decimal. eg 246.0 -> 246
                value1 = value1.split('.')[0] 

            return value1.replace(" ", '') == value2.replace(" ", '')

        else:
            try:
                return value1.lower().replace(" ", '') == value2.lower().replace(" ", '')
            except:
                return value1 == value2

    except:
        print(column_name, value1, type(value1), value2, type(value2))


In [136]:
date_columns = ['case_rec_date', 'renewal_date', 'response_by']

In [137]:
for column in date_columns:
    comparison[column] = comparison[column].astype(str).apply(format_comparison_date)

In [138]:
for column in date_columns:
    golden[column] = golden[column].apply(parse_and_format_golden_date)

non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value in golden  NA
non date value

In [139]:
def echo(*args): # for debugging lambda arguments
    # print(*args)
    return True

# Create an empty dataframe to store the results
result_data = {'case': []}
result_df = pd.DataFrame(result_data)
validated_df = pd.DataFrame(result_data)

# Iterate through columns and rows
for column in columns_to_check:    
    result_df[column] = comparison.apply( # Comparing all values against goldens 
        lambda row: custom_callback(column, row[column], golden.loc[row['case'] == golden['case']][column]),
        axis=1)
    try:
        validated_df[column] = comparison.apply( # Comparing values against goldens IF has passed validation
            lambda row: 
            custom_callback(column, row[column], golden.loc[row['case'] == golden['case']][column])            
            if echo(row.case) and validations.loc[validations['case'] == row['case']][column].values[0] else 'fail', # only three possible values
            axis=1)# True, False, fail (for fields that didn't pass validation)
    except:
        print('1')

# Add the 'case' column to the result dataframe
result_df['case'] = comparison['case']
validated_df['case'] = validations['case']

date err renewal_date 30/06/203 30/06/2023 False time data '30/06/203' does not match format '%d/%m/%Y'
date err renewal_date 13/06/2023 30/06/2023 False time data '13/06/2023' does not match format '%m/%d/%Y'
date err renewal_date 31/05/2020 31/05/2023 False time data '31/05/2020' does not match format '%m/%d/%Y'
date err renewal_date 23/06/2019 19/06/2023 False time data '23/06/2019' does not match format '%m/%d/%Y'
date err renewal_date 20/06/YYYY 20/06/2023 False time data '20/06/YYYY' does not match format '%d/%m/%Y'
date err renewal_date 15/05/2023 24/05/2023 False time data '15/05/2023' does not match format '%m/%d/%Y'
date err renewal_date 14/06/2023 31/05/2023 False time data '14/06/2023' does not match format '%m/%d/%Y'
date err renewal_date 17/06/2023 16/06/2023 False time data '17/06/2023' does not match format '%m/%d/%Y'
date err renewal_date 02/06/YYYY 02/06/2023 False time data '02/06/YYYY' does not match format '%d/%m/%Y'
date err renewal_date 23/06/2023 30/06/2023 Fals

In [140]:
result_df.head()

,case,broker_name,broker_email,broker_telephone,broker_postcode,broker_agency_name,customer_name,customer_postcode,business_description,trade_description,renewal_date,target_premium,response_by,current_insurer,product,existing_broker,case_rec_date
0,201512.0,True,True,True,True,True,True,False,False,False,False,True,True,True,False,False,True
1,201513.0,True,True,False,True,False,True,True,True,False,True,True,False,True,True,False,True
2,201514.0,True,True,True,True,False,True,True,False,True,True,True,False,True,True,False,True
3,201515.0,True,True,True,True,True,True,True,True,False,True,True,True,True,True,False,True
4,201516.0,True,True,True,True,False,True,True,True,False,True,True,False,True,False,False,True


In [141]:
def get_field_level_accuracy(target, fields=None):

    if not fields:
        fields = target.columns
    
    result = {}

    for field in fields:
        counts = target[field].value_counts()
        total_counts = counts.sum()
        true_fields = counts.get(True, 0)
        false_fields = counts.get(False, 0)

        result[field] = round(true_fields/total_counts, 3)
        print(field, "total", total_counts, "true", true_fields)

    return result


def get_total_field_accuracy(target, fields=None):

    if not fields:
        fields = target.columns
    
    total_eligible_fields = 0
    total_matches = 0

    for field in fields:
        counts = target[field].value_counts()
        total_eligible_fields += counts.sum()
        total_matches += counts.get(True, 0)

    return round(total_matches / total_eligible_fields, 3)


def get_validated_field_level_accuracy(target, fields=None):

    if not fields:
        fields = target.columns
    
    result = {}
    total = 0
    for field in fields:
        counts = target[field].value_counts()
        total_counts = counts.sum()
        true_fields = counts.get(True, 0) # Fields that passed validation and are accurate against the Goldens
        not_applicable_fields = counts.get("fail", 0) # Fields that failed Validation
        
        value = round(true_fields/(total_counts - not_applicable_fields), 3)
        result[field] = value
        total += value

    result['avg'] = round( total / len(fields) , 3 )
    return result


def get_field_automation_rate(target, fields=None):

    if not fields:
        fields = target.columns
    
    result = {}
    total = 0
    for field in fields:
        
        counts = target[field].value_counts()
        total_counts = counts.sum()
        true_fields = counts.get(True, 0) # Fields that passes validation
        false_fields = counts.get(False, 0) # Fields that passed validation but are incaccurate
        value = round((true_fields + false_fields)/total_counts, 3)
        result[field] = value
        total += value

    result['avg'] = round(total / len(fields), 3)
    return result


def get_false_positives(target, fields=None):
    
    if not fields:
        fields = target.columns
    
    result = {}
    total = 0
    for field in fields:
        counts = target[field].value_counts()
        total_counts = counts.sum()
        false_fields = counts.get(False, 0) # Fields that passed validation and are accurate against the Goldens
        # not_applicable_fields = counts.get("fail", 0) # Fields that failed Validation
        
        value = round(false_fields/total_counts , 5)
        result[field] = value
        total += value

    result['avg'] = round( total / len(fields) , 3 )
    return result    


In [142]:
validated_df

,case,broker_name,broker_email,broker_telephone,broker_postcode,broker_agency_name,customer_name,customer_postcode,business_description,trade_description,renewal_date,target_premium,response_by,current_insurer,product,existing_broker,case_rec_date
0,201512.0,True,True,True,fail,True,True,fail,False,False,fail,True,fail,True,False,False,fail
1,201513.0,True,True,False,fail,fail,True,True,True,fail,True,fail,fail,fail,True,fail,fail
2,201514.0,True,True,True,fail,fail,fail,True,False,fail,True,fail,fail,True,True,fail,True
3,201515.0,True,True,True,fail,fail,True,True,True,fail,True,True,fail,fail,True,fail,fail
4,201516.0,fail,fail,fail,True,False,True,True,True,fail,fail,fail,fail,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,201792.0,True,True,True,fail,fail,False,True,False,fail,True,fail,True,False,True,False,True
262,201793.0,True,True,True,fail,fail,True,fail,True,fail,True,True,True,fail,True,True,fail
263,201794.0,True,True,True,fail,True,True,fail,False,fail,fail,True,fail,False,fail,True,True
264,201795.0,True,True,True,fail,fail,True,True,False,fail,fail,fail,fail,False,False,False,fail


In [143]:
accuracy = {
    "total_FA": get_total_field_accuracy(result_df, columns_to_check),
    "FLA": get_field_level_accuracy(result_df, columns_to_check),
    
    "validated_FLA": get_validated_field_level_accuracy(validated_df, columns_to_check),
    "field_automation_rate": get_field_automation_rate(validated_df, columns_to_check),
    "FP": get_false_positives(validated_df, columns_to_check)
}

broker_name total 266 true 258
broker_email total 266 true 257
broker_telephone total 266 true 222
broker_postcode total 266 true 236
broker_agency_name total 266 true 188
customer_name total 266 true 238
customer_postcode total 266 true 220
business_description total 266 true 194
trade_description total 266 true 94
renewal_date total 266 true 205
target_premium total 266 true 224
response_by total 266 true 78
current_insurer total 266 true 210
product total 266 true 152
existing_broker total 266 true 54
case_rec_date total 266 true 248


In [144]:
# import json
pretty_printed_dict = json.dumps(accuracy, indent=2)

print(pretty_printed_dict)

{
  "total_FA": 0.723,
  "FLA": {
    "broker_name": 0.97,
    "broker_email": 0.966,
    "broker_telephone": 0.835,
    "broker_postcode": 0.887,
    "broker_agency_name": 0.707,
    "customer_name": 0.895,
    "customer_postcode": 0.827,
    "business_description": 0.729,
    "trade_description": 0.353,
    "renewal_date": 0.771,
    "target_premium": 0.842,
    "response_by": 0.293,
    "current_insurer": 0.789,
    "product": 0.571,
    "existing_broker": 0.203,
    "case_rec_date": 0.932
  },
  "validated_FLA": {
    "broker_name": 0.97,
    "broker_email": 0.977,
    "broker_telephone": 0.869,
    "broker_postcode": 0.977,
    "broker_agency_name": 0.754,
    "customer_name": 0.908,
    "customer_postcode": 0.914,
    "business_description": 0.772,
    "trade_description": 0.0,
    "renewal_date": 0.864,
    "target_premium": 0.913,
    "response_by": 0.47,
    "current_insurer": 0.807,
    "product": 0.74,
    "existing_broker": 0.268,
    "case_rec_date": 0.942,
    "avg": 0.75

In [148]:
sorted_order = ['broker_agency_name','broker_email', 'broker_name', 'broker_postcode','broker_telephone', 'business_description',
        'case_rec_date',
'current_insurer',
'customer_name',
'customer_postcode', 'renewal_date', 'product', 'response_by',
'existing_broker',
'target_premium',
'trade_description']
# order.remove("broker_type")
for column in sorted_order:
    print(accuracy['FP'][column])

0.12782
0.02256
0.03008
0.00752
0.12406
0.18797
0.04135
0.16541
0.08271
0.05263
0.07895
0.18797
0.13158
0.54511
0.04887
0.01504


In [149]:
validated_df.tail()

,case,broker_name,broker_email,broker_telephone,broker_postcode,broker_agency_name,customer_name,customer_postcode,business_description,trade_description,renewal_date,target_premium,response_by,current_insurer,product,existing_broker,case_rec_date
261,201792.0,True,True,True,fail,fail,False,True,False,fail,True,fail,True,False,True,False,True
262,201793.0,True,True,True,fail,fail,True,fail,True,fail,True,True,True,fail,True,True,fail
263,201794.0,True,True,True,fail,True,True,fail,False,fail,fail,True,fail,False,fail,True,True
264,201795.0,True,True,True,fail,fail,True,True,False,fail,fail,fail,fail,False,False,False,fail
265,201796.0,False,False,fail,fail,fail,True,fail,False,fail,True,fail,False,fail,True,fail,False


In [150]:
validations.columns

Index(['broker_name', 'broker_email', 'broker_telephone', 'broker_postcode',
       'broker_agency_name', 'customer_name', 'customer_postcode',
       'business_description', 'trade_description', 'renewal_date',
       'target_premium', 'response_by', 'current_insurer', 'product',
       'existing_broker', 'case_rec_date', 'case'],
      dtype='object')

In [151]:
merged = pd.merge(pd.merge(comparison, golden, on='case', how='inner', suffixes=('', '_expected')), 
result_df, on='case', how='inner', suffixes=('', '_matched'))

validated_copy = validations.applymap(lambda x: 'PASS' if x == True else 'FAIL')
validated_copy['case'] = validations['case']

merged = pd.merge(validated_copy, merged, on='case', suffixes=('_valid', ''))

# merged = merged.drop(columns="Sl No.")
sorted_columns = ['case'] + sorted(merged.columns[1:])

merged = merged[sorted_columns]
merged.to_csv('./out/2411005_hybrid.csv', index=False)

/var/folders/v2/0rlhnypj2x96mlk2_hnygbbh0000gn/T/ipykernel_61847/3494481980.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  validated_copy = validations.applymap(lambda x: 'PASS' if x == True else 'FAIL')


In [152]:
validated_broker_name = validated_copy[['broker_name', 'case']]


In [153]:
validated_broker_name.columns

Index(['broker_name', 'case'], dtype='object')

In [154]:
merged_broker_name = merged[['broker_name', 'broker_name_expected', 'broker_name_matched', 'case']]

merged_broker_name = merged_broker_name.loc[:,~merged_broker_name.columns.duplicated()]

In [155]:
merged_broker_name.columns

Index(['broker_name', 'broker_name_expected', 'broker_name_matched', 'case'], dtype='object')

In [156]:
merged_broker_name = pd.merge(validated_broker_name, merged_broker_name, on='case', suffixes=('_valid', ''))

In [157]:

sorted_columns = sorted(merged_broker_name.columns)

sorted_columns
merged_broker_name = merged_broker_name[sorted_columns]

In [158]:
merged_broker_name

,broker_name,broker_name_expected,broker_name_matched,broker_name_valid,case
0,Dan Davis,Dan Davis,True,PASS,201512.0
1,Emma Parker,Emma Parker,True,PASS,201513.0
2,Andrea Keenan,Andrea Keenan,True,PASS,201514.0
3,Lisa Forshaw,Lisa Forshaw,True,PASS,201515.0
4,Abigail Hale,Abigail Hale,True,FAIL,201516.0
...,...,...,...,...,...
261,Yvonne Ehinmoti,Yvonne Ehinmoti,True,PASS,201792.0
262,Matt Capon,Matt Capon,True,PASS,201793.0
263,Ian Henehan,Ian Henehan,True,PASS,201794.0
264,Isaac Lowe,Isaac Lowe,True,PASS,201795.0


In [159]:
merged_broker_name.to_csv('./out/241105_hybrid_broker_name.csv', index=False)